In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import joblib as joblib
import glob

from time import strptime
from datetime import datetime

In [2]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", "{:,.2f}".format)

## 1. Importing data

In [3]:
DATA_PATH = "../data/"

In [4]:
df = pd.read_csv(os.path.join(DATA_PATH, "bank-additional-full.csv"), sep=";")

In [5]:
df.shape

(41188, 22)

In [6]:
df

,age,job,marital,education,default,housing,loan,contact,month,year,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,2008,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no
1,57,services,married,high.school,unknown,no,no,telephone,may,2008,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no
2,37,services,married,high.school,no,yes,no,telephone,may,2008,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,2008,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no
4,56,services,married,high.school,no,no,yes,telephone,may,2008,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,2010,fri,334,1,999,0,nonexistent,-1.10,94.77,-50.80,1.03,"4,963.60",yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,2010,fri,383,1,999,0,nonexistent,-1.10,94.77,-50.80,1.03,"4,963.60",no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,2010,fri,189,2,999,0,nonexistent,-1.10,94.77,-50.80,1.03,"4,963.60",no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,2010,fri,442,1,999,0,nonexistent,-1.10,94.77,-50.80,1.03,"4,963.60",yes


Let's build `anomes` variable

In [7]:
df["month_nb"] = df["month"].apply(lambda x : strptime(x,'%b').tm_mon)
df["anomes"] = (df["year"].astype(str) + df["month_nb"].astype(str).str.zfill(2)).astype(int)

In [8]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,year,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_nb,anomes
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,2008,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
1,57,services,married,high.school,unknown,no,no,telephone,may,2008,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
2,37,services,married,high.school,no,yes,no,telephone,may,2008,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
3,40,admin.,married,basic.6y,no,no,no,telephone,may,2008,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
4,56,services,married,high.school,no,no,yes,telephone,may,2008,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805


Now, we see the target distribution

In [9]:
df["y"].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [10]:
df_aux = df.groupby(["anomes"])["y"].value_counts().rename("counts").to_frame().reset_index()
df_aux = pd.pivot_table(df_aux, values="counts", index="anomes", columns="y")
df_aux["%yes"] = df_aux["yes"] / (df_aux["yes"] + df_aux["no"])
df_aux["total"] = df_aux["yes"] + df_aux["no"]
df_aux = df_aux.reset_index()
df_aux.sort_values(by="anomes")

y,anomes,no,yes,%yes,total
0,200805,7523,240,0.03,7763
1,200806,4186,188,0.04,4374
2,200807,6278,407,0.06,6685
3,200808,4904,271,0.05,5175
4,200810,25,42,0.63,67
5,200811,3426,190,0.05,3616
6,200812,9,1,0.10,10
7,200903,156,126,0.45,282
8,200904,2016,442,0.18,2458
9,200905,5270,524,0.09,5794


We see really asymetric distribution of registers across the months

## 2. Feature Engineering

We will create variables that are available in the prediction time, and also that would not bias the model towards personal information

Obs.: Let's create an index to identify each call / client.

In [11]:
df_feat = df.copy()

In [12]:
df_feat = df_feat.reset_index().rename(columns={"index":"id_call"}).set_index("id_call")
df_feat.head()

,age,job,marital,education,default,housing,loan,contact,month,year,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,month_nb,anomes
id_call,,,,,,,,,,,,,,,,,,,,,,,,
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,2008,mon,261,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
1,57,services,married,high.school,unknown,no,no,telephone,may,2008,mon,149,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
2,37,services,married,high.school,no,yes,no,telephone,may,2008,mon,226,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
3,40,admin.,married,basic.6y,no,no,no,telephone,may,2008,mon,151,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805
4,56,services,married,high.school,no,no,yes,telephone,may,2008,mon,307,1,999,0,nonexistent,1.10,93.99,-36.40,4.86,"5,191.00",no,5,200805


In [13]:
df_feat.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'year', 'day_of_week', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'month_nb', 'anomes'],
      dtype='object')

### 2.1 Social and economic context attributes

We can use these attributes with no risk to bias the model (exception to the variable `nr.employed`).

In [14]:
df_feat = df_feat.drop(columns=["nr.employed"])

In [15]:
df_feat[['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m']].head()

,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m
id_call,,,,
0,1.10,93.99,-36.40,4.86
1,1.10,93.99,-36.40,4.86
2,1.10,93.99,-36.40,4.86
3,1.10,93.99,-36.40,4.86
4,1.10,93.99,-36.40,4.86


In [16]:
df_feat[['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m']].dtypes

emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
dtype: object

In [17]:
df_feat[['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m']].isna().sum()

emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
dtype: int64

As we can see, no need to transformn these variables.

### 2.2 Other attributes

Let's use the variables about past calls and campaigns

In [18]:
df_feat[["pdays", "previous", "poutcome", "campaign"]].head()

,pdays,previous,poutcome,campaign
id_call,,,,
0,999,0,nonexistent,1
1,999,0,nonexistent,1
2,999,0,nonexistent,1
3,999,0,nonexistent,1
4,999,0,nonexistent,1


In [19]:
df_feat[["pdays", "previous", "poutcome", "campaign"]].dtypes

pdays        int64
previous     int64
poutcome    object
campaign     int64
dtype: object

In [20]:
df_feat["poutcome"].value_counts()

nonexistent    35563
failure         4252
success         1373
Name: poutcome, dtype: int64

In [21]:
def transform_poutcome(poutcome):
    poutcome_value = -1.0
    if poutcome == "success":
        poutcome_value = 1.0
    else: # "no" or "unknown"
        poutcome_value = 0.0
    return poutcome_value

df_feat["poutcome"] = df_feat["poutcome"].apply(transform_poutcome)

In [22]:
df_feat["poutcome"].value_counts()

0.00    39815
1.00     1373
Name: poutcome, dtype: int64

In [23]:
df_feat[["pdays", "previous", "poutcome", "campaign"]].describe()

,pdays,previous,poutcome,campaign
count,"41,188.00","41,188.00","41,188.00","41,188.00"
mean,962.48,0.17,0.03,2.57
std,186.91,0.49,0.18,2.77
min,0.00,0.00,0.00,1.00
25%,999.00,0.00,0.00,1.00
50%,999.00,0.00,0.00,2.00
75%,999.00,0.00,0.00,3.00
max,999.00,7.00,1.00,56.00


In [24]:
df_feat[["pdays", "previous", "poutcome", "campaign"]].isna().sum()

pdays       0
previous    0
poutcome    0
campaign    0
dtype: int64

### 2.3 Related with the last contact of the current campaign

The variables in this section are `contact`, `month`, `day_of_week`, `duration` (ane `year`, created manually).

We won't use `month`, becaus of the asymetry in the quantity of example per month. Neither `duration`, becaus we don't have it in prediction time.

Let's incorporate `contact` and `day_of_week` to the model.

In [25]:
df_feat = df_feat.drop(columns=["month", "year", "duration"])

In [26]:
df_feat["contact"].value_counts()

cellular     26144
telephone    15044
Name: contact, dtype: int64

In [27]:
def transform_contact(contact):
    contact_value = -1.0
    if contact == "cellular":
        contact_value = 1.0
    else: # "no" or "unknown"
        contact_value = 0.0
    return contact_value

df_feat["contact"] = df_feat["contact"].apply(transform_contact)
df_feat = df_feat.rename(columns={"contact":"contacted_by_cellular"})

In [28]:
df_feat["contacted_by_cellular"].value_counts()

1.00    26144
0.00    15044
Name: contacted_by_cellular, dtype: int64

In [29]:
df_feat["contacted_by_cellular"].isna().sum()

0

In [30]:
df_feat["day_of_week"].value_counts()

thu    8623
mon    8514
wed    8134
tue    8090
fri    7827
Name: day_of_week, dtype: int64

In [31]:
enc_dow = df_feat["day_of_week"].str.get_dummies()
enc_dow.columns = "day_of_week_" + enc_dow.columns
df_feat = pd.concat([df_feat, enc_dow], axis=1)
df_feat = df_feat.drop(columns=["day_of_week", "day_of_week_thu"])
df_feat.head()

,age,job,marital,education,default,housing,loan,contacted_by_cellular,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,y,month_nb,anomes,day_of_week_fri,day_of_week_mon,day_of_week_tue,day_of_week_wed
id_call,,,,,,,,,,,,,,,,,,,,,,,
0,56,housemaid,married,basic.4y,no,no,no,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
1,57,services,married,high.school,unknown,no,no,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
2,37,services,married,high.school,no,yes,no,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
3,40,admin.,married,basic.6y,no,no,no,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
4,56,services,married,high.school,no,no,yes,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0


### 2.4 Bank client data

We will avoid using personal information like `age`, `job`, `marital`, `education` avoiding any bias in selecting the public. Using any of these variables would require business validation of the compliance rules of the enterprise, in order to avoid excluding public that should be proportionaly called.

Let's use `default`, `housing`, `loan`.

In [32]:
df_feat = df_feat.drop(columns=["age", "job", "marital", "education"])

In [33]:
def transform_yes_no_column(default):
    default_value = -1.0
    if default == "yes":
        default_value = 1.0
    else: # "no" or "unknown"
        default_value = 0.0
    return default_value

df_feat["default"] = df_feat["default"].apply(transform_yes_no_column)
df_feat["housing"] = df_feat["housing"].apply(transform_yes_no_column)
df_feat["loan"] = df_feat["loan"].apply(transform_yes_no_column)

In [34]:
df_feat

,default,housing,loan,contacted_by_cellular,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,y,month_nb,anomes,day_of_week_fri,day_of_week_mon,day_of_week_tue,day_of_week_wed
id_call,,,,,,,,,,,,,,,,,,,
0,0.00,0.00,0.00,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
1,0.00,0.00,0.00,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
2,0.00,1.00,0.00,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
3,0.00,0.00,0.00,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
4,0.00,0.00,1.00,0.00,1,999,0,0.00,1.10,93.99,-36.40,4.86,no,5,200805,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,0.00,1.00,0.00,1.00,1,999,0,0.00,-1.10,94.77,-50.80,1.03,yes,11,201011,1,0,0,0
41184,0.00,0.00,0.00,1.00,1,999,0,0.00,-1.10,94.77,-50.80,1.03,no,11,201011,1,0,0,0
41185,0.00,1.00,0.00,1.00,2,999,0,0.00,-1.10,94.77,-50.80,1.03,no,11,201011,1,0,0,0


### 2.5 Response variable

In [35]:
df_feat["y"].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [36]:
df_feat["y"] = df_feat["y"].apply(transform_yes_no_column)

In [37]:
df_feat["y"].value_counts()

0.00    36548
1.00     4640
Name: y, dtype: int64

## 3. Data Prep

Are there nulls in the resulting database?

In [38]:
df_feat.isna().sum().sort_values(ascending=False).head(10)

default            0
cons.conf.idx      0
day_of_week_tue    0
day_of_week_mon    0
day_of_week_fri    0
anomes             0
month_nb           0
y                  0
euribor3m          0
cons.price.idx     0
dtype: int64

## 4. Train and test division

Let's do out-of-time division, to avoid using past and future examples in training.

In [41]:
df_aux = df_feat.groupby(["anomes"])["y"].value_counts().rename("counts").to_frame().reset_index()
df_aux = pd.pivot_table(df_aux, values="counts", index="anomes", columns="y")
df_aux["%1.0"] = df_aux[1] / (df_aux[1] + df_aux[0])
df_aux["total"] = df_aux[1] + df_aux[0]
df_aux = df_aux.reset_index()
df_aux.sort_values(by="anomes")

y,anomes,0.0,1.0,%1.0,total
0,200805,7523,240,0.03,7763
1,200806,4186,188,0.04,4374
2,200807,6278,407,0.06,6685
3,200808,4904,271,0.05,5175
4,200810,25,42,0.63,67
5,200811,3426,190,0.05,3616
6,200812,9,1,0.10,10
7,200903,156,126,0.45,282
8,200904,2016,442,0.18,2458
9,200905,5270,524,0.09,5794


In [42]:
df_train = df_feat[df_feat["anomes"] <= 200812]
df_test = df_feat[(df_feat["anomes"] >= 200812) & (df_feat["anomes"] <= 200912)]

In [43]:
df_train["y"].value_counts(), df_test["y"].value_counts()

(0.00    26351
 1.00     1339
 Name: y, dtype: int64,
 0.00    9221
 1.00    2229
 Name: y, dtype: int64)

Target proportion in train and test

In [45]:
len(df_train[df_train["y"] == 1.0]) / len(df_train), len(df_test[df_test["y"] == 1.0]) / len(df_test)

(0.04835680751173709, 0.19467248908296944)

Shape of train and test

In [47]:
df_train.shape, df_test.shape

((27690, 19), (11450, 19))

## 4. Feature Selection